In [4]:
import pandas as pd 
#Last downloaded on september 10th
terrorism = pd.read_excel("../Databases/Global Terrorism Database/globalterrorismdb_0919dist.xlsx")
terrorism = pd.DataFrame(terrorism)


In [17]:
targets = terrorism[[ 'iyear',
 'imonth',
 'iday',
 'approxdate',
 'extended',
 'resolution',
 'country',
 'country_txt',
 'region',
 'region_txt','targtype1',
 'targtype1_txt',
 'targsubtype1',
 'targsubtype1_txt',
  'latitude',
 'longitude',]]

In [30]:
oil = targets.loc[(targets["targsubtype1"] == 59) | (targets["targsubtype1"] == 108) | (targets["targsubtype1"] == 1)]

In [32]:
oil.head()

,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt
129,1,Business,1.0,Gas/Oil/Electric
422,21,Utilities,108.0,Oil
495,1,Business,1.0,Gas/Oil/Electric
765,21,Utilities,108.0,Oil
792,21,Utilities,108.0,Oil


In [18]:
targets.head(10)

,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt
0,14,Private Citizens & Property,68.0,Named Civilian
1,7,Government (Diplomatic),45.0,"Diplomatic Personnel (outside of embassy, cons..."
2,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility
3,7,Government (Diplomatic),46.0,Embassy/Consulate
4,7,Government (Diplomatic),46.0,Embassy/Consulate
5,3,Police,22.0,"Police Building (headquarters, station, school)"
6,3,Police,25.0,Police Security Forces/Officers
7,21,Utilities,107.0,Electricity
8,4,Military,28.0,Military Recruiting Station/Academy
9,2,Government (General),21.0,Government Building/Facility/Office


In [6]:
terrorism.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaT,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaT,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaT,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaT,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaT,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [11]:
cols = terrorism.columns.tolist()
cols

['eventid',
 'iyear',
 'imonth',
 'iday',
 'approxdate',
 'extended',
 'resolution',
 'country',
 'country_txt',
 'region',
 'region_txt',
 'provstate',
 'city',
 'latitude',
 'longitude',
 'specificity',
 'vicinity',
 'location',
 'summary',
 'crit1',
 'crit2',
 'crit3',
 'doubtterr',
 'alternative',
 'alternative_txt',
 'multiple',
 'success',
 'suicide',
 'attacktype1',
 'attacktype1_txt',
 'attacktype2',
 'attacktype2_txt',
 'attacktype3',
 'attacktype3_txt',
 'targtype1',
 'targtype1_txt',
 'targsubtype1',
 'targsubtype1_txt',
 'corp1',
 'target1',
 'natlty1',
 'natlty1_txt',
 'targtype2',
 'targtype2_txt',
 'targsubtype2',
 'targsubtype2_txt',
 'corp2',
 'target2',
 'natlty2',
 'natlty2_txt',
 'targtype3',
 'targtype3_txt',
 'targsubtype3',
 'targsubtype3_txt',
 'corp3',
 'target3',
 'natlty3',
 'natlty3_txt',
 'gname',
 'gsubname',
 'gname2',
 'gsubname2',
 'gname3',
 'gsubname3',
 'motive',
 'guncertain1',
 'guncertain2',
 'guncertain3',
 'individual',
 'nperps',
 'nperpcap',
 